In [ ]:
# Values that might change:

MOVIE_ID =  "KOK_20240219_BUV_KOK_020_01.mp4"
EXPECTED_CLIP_AMOUNT = 60


survey = MOVIE_ID[:16]
deployment = MOVIE_ID[:-4]

clips_folder = f"/nesi/project/wildlife03546/code/kso_upload_copy/notebooks/classify/{deployment}.mp4_zooniverseclips"
s3_biigle_folder = f"process_files/clips/biigle/{survey}/{deployment}/"


In [ ]:

%load_ext autoreload
%autoreload 2
import re

from pathlib import Path


from sftk.common import S3_BUCKET, BIIGLE_API_EMAIL, BIIGLE_API_TOKEN, BIIGLE_DISK_ID, BIIGLE_PROJECT_ID
from sftk.s3_handler import S3Handler
from sftk.external.biigle_api import Api



In [ ]:
s3_handler = S3Handler()

In [ ]:
def find_files_with_extension(folder_path, extension):
    return [str(f) for f in Path(folder_path).glob(f"*.{extension}")]

In [ ]:
# Get clips 
clips_files = find_files_with_extension(clips_folder, "mp4")
# Sort by the number after '_clip_'
sorted_paths = sorted(clips_files, key=lambda x: int(re.search(r'_clip_(\d+)_', x).group(1)))


In [ ]:


clip_names = []
for file in sorted_paths:
    file_name = Path(file).name  
    s3_path = Path(s3_biigle_folder) / file_name
    clip_names.append(file_name)

    s3_handler.upload_file_to_s3(file, key=str(s3_path), bucket=S3_BUCKET, content_type="video/mp4")

In [ ]:
mp4_files = s3_handler.get_file_paths_set_from_s3(bucket=S3_BUCKET, prefix = s3_biigle_folder, suffixes= (".mp4",), file_names_only=True)


In [ ]:
# check if local files and files in s3 match, and there are 60 of them
assert set(clip_names) == mp4_files
assert len(mp4_files) == EXPECTED_CLIP_AMOUNT

In [ ]:
api = Api(BIIGLE_API_EMAIL, BIIGLE_API_TOKEN)

In [ ]:
# Create pending volume
pending_volume_response = api.post(f"projects/{BIIGLE_PROJECT_ID}/pending-volumes", json={"media_type":"video"})
pending_volume_info = pending_volume_response.json()

# If the above fails: 
# One can have only one pending volume at a time. 
# You can find your pending volume id in gui by trying to make a new volume, (which will fail and take you to the existing pending volume) and the id is in the url 


In [ ]:
# pending_volume_id = 1234 # add your own if fetching pending_volume_info above fails
pending_volume_id = pending_volume_info["id"]

In [ ]:

s3_url = f"disk-{BIIGLE_DISK_ID}://{s3_biigle_folder}"

fill_pv = api.put(f"pending-volumes/{pending_volume_id}", 
                  json={"name": deployment , 
                        "url": s3_url, 
                        "files": clip_names})
fill_pv.json()